# 🔄 Week 8: Reranking for Improved Retrieval

This notebook covers reranking techniques to improve retrieval quality.

## Table of Contents
1. [Why Reranking?](#1-why-reranking)
2. [Cross-Encoder Reranking](#2-cross-encoder-reranking)
3. [LLM-Based Reranking](#3-llm-based-reranking)
4. [Reciprocal Rank Fusion](#4-reciprocal-rank-fusion)
5. [Diversity Reranking (MMR)](#5-diversity-reranking-mmr)
6. [Building a Reranking Pipeline](#6-building-a-reranking-pipeline)

---

## 1. Why Reranking?

### 1.1 The Two-Stage Retrieval Paradigm

```
Query → [Stage 1: Fast Retrieval] → Top-100 → [Stage 2: Reranking] → Top-10
              (BM25, Dense)                    (Cross-Encoder, LLM)
```

**Why Two Stages?**

| Stage | Speed | Accuracy | Use Case |
|-------|-------|----------|----------|
| Initial Retrieval | ⚡ Fast | Good | Narrow down from millions |
| Reranking | 🐢 Slow | Excellent | Refine top candidates |

### 1.2 Reranking Methods Overview

| Method | How It Works | Pros | Cons |
|--------|--------------|------|------|
| **Cross-Encoder** | BERT scores query-doc pairs | Most accurate | Slow |
| **LLM Reranking** | Prompt LLM to rank | Zero-shot, semantic | Expensive |
| **RRF** | Combine multiple rankings | Simple, effective | Needs multiple rankings |
| **MMR** | Maximize relevance & diversity | Reduces redundancy | Needs embeddings |

In [ ]:
# Setup
import sys
sys.path.insert(0, '../..')

import numpy as np
from dataclasses import dataclass
from typing import List, Dict, Optional

# Import our reranking module
from src.reranking import (
    CrossEncoderReranker,
    LLMReranker,
    ReciprocalRankFusion,
    DiversityReranker,
    RerankConfig,
)
from src.reranking.reranking import Document

print("✅ Setup complete!")

---

## 2. Cross-Encoder Reranking

### 2.1 Understanding Cross-Encoders

**Bi-Encoder (Initial Retrieval)**:
- Encodes query and document separately
- Compare via dot product
- Fast but limited interaction

```
Query → [Encoder] → q_vec
                            → Dot Product → Score
Doc   → [Encoder] → d_vec
```

**Cross-Encoder (Reranking)**:
- Processes query and document together
- Deep cross-attention between tokens
- Rich interaction, more accurate

```
[CLS] query tokens [SEP] document tokens [SEP] → [BERT] → Score
```

### 2.2 Implementation

In [ ]:
# Sample documents (simulating initial retrieval results)
documents = [
    Document(id="1", content="Machine learning is a subset of artificial intelligence.", score=0.85),
    Document(id="2", content="Deep learning uses neural networks with many layers.", score=0.82),
    Document(id="3", content="Natural language processing helps computers understand text.", score=0.78),
    Document(id="4", content="Neural networks are inspired by the human brain.", score=0.75),
    Document(id="5", content="Python is popular for data science applications.", score=0.70),
]

query = "How do neural networks learn from data?"

# Initialize cross-encoder reranker
cross_encoder = CrossEncoderReranker()

# Rerank
results = cross_encoder.rerank(query, documents, top_k=3)

print(f"Query: '{query}'\n")
print("Cross-Encoder Reranking Results:")
print("=" * 60)

for r in results:
    rank_change = r.original_rank - r.final_rank
    change_str = f"(+{rank_change})" if rank_change > 0 else f"({rank_change})" if rank_change < 0 else "(=)"
    
    print(f"\nRank {r.final_rank} {change_str}: {r.document.content}")
    print(f"  Original Score: {r.original_score:.3f} → Rerank Score: {r.rerank_score:.3f}")

### 2.3 When to Use Cross-Encoders

✅ **Good for:**
- Search quality is critical
- Small candidate set (< 100 documents)
- Complex queries requiring deep understanding

❌ **Not good for:**
- Real-time with many documents
- Very fast response requirements
- Initial retrieval from large corpus

---

## 3. LLM-Based Reranking

### 3.1 Concept

Use a large language model to assess relevance through prompting.

**Strategies:**

| Strategy | Description | Example |
|----------|-------------|----------|
| **Pointwise** | Score each doc independently | "Rate 0-10..." |
| **Pairwise** | Compare two docs | "Which is more relevant?" |
| **Listwise** | Rank entire list | "Order by relevance..." |

### 3.2 Pointwise Prompt Example

```
Given the query and document, rate relevance from 0-10.

Query: How do neural networks learn?
Document: Deep learning uses neural networks with many layers.

Relevance (0-10): 8
```

In [ ]:
# LLM Reranker (conceptual - uses fallback without actual LLM)
llm_reranker = LLMReranker(strategy="pointwise")

# Demonstrate the concept
print("LLM-Based Reranking Concept")
print("=" * 60)
print("""
In production, the LLM would receive prompts like:

┌─────────────────────────────────────────────────────────┐
│ Given the following query and document, rate the       │
│ relevance on a scale of 0-10.                          │
│                                                        │
│ Query: How do neural networks learn from data?         │
│                                                        │
│ Document: Deep learning uses neural networks with      │
│ many layers.                                           │
│                                                        │
│ Relevance score (0-10):                                │
└─────────────────────────────────────────────────────────┘

The LLM would respond with a score, which is then used
to rerank the documents.
""")

---

## 4. Reciprocal Rank Fusion

### 4.1 The RRF Formula

RRF combines multiple rankings without needing score normalization:

$$RRF(d) = \sum_{r \in R} \frac{1}{k + rank_r(d)}$$

Where:
- $d$ = document
- $R$ = set of rankings
- $k$ = constant (usually 60)
- $rank_r(d)$ = rank of document d in ranking r

### 4.2 Why RRF Works

| Property | Explanation |
|----------|-------------|
| **No normalization** | Ranks are comparable across systems |
| **Diminishing returns** | Top ranks contribute more |
| **Robust** | One bad ranker doesn't dominate |

In [ ]:
# RRF Example
rrf = ReciprocalRankFusion(k=60)

# Simulate different retrieval rankings
# BM25 ranking
bm25_docs = [
    Document(id="1", content="Machine learning is AI.", score=0.9),
    Document(id="2", content="Deep learning uses neural nets.", score=0.8),
    Document(id="3", content="NLP processes text.", score=0.7),
    Document(id="4", content="Neural networks learn patterns.", score=0.6),
]

# Dense retrieval ranking (different order)
dense_docs = [
    Document(id="4", content="Neural networks learn patterns.", score=0.85),
    Document(id="2", content="Deep learning uses neural nets.", score=0.82),
    Document(id="1", content="Machine learning is AI.", score=0.75),
    Document(id="3", content="NLP processes text.", score=0.70),
]

# Apply RRF
fused_results = rrf.fuse([bm25_docs, dense_docs], top_k=4)

print("Reciprocal Rank Fusion Results")
print("=" * 60)
print("\nBM25 Ranking:   [1, 2, 3, 4]")
print("Dense Ranking:  [4, 2, 1, 3]")
print("\nFused Ranking:")

for r in fused_results:
    print(f"  Rank {r.final_rank}: Doc {r.document.id} (RRF Score: {r.rerank_score:.4f})")

# Explain RRF calculation
print("\nRRF Score Calculation (k=60):")
print("-" * 40)
print("Doc 2: 1/(60+2) + 1/(60+2) = 0.0323")
print("Doc 1: 1/(60+1) + 1/(60+3) = 0.0322")
print("Doc 4: 1/(60+4) + 1/(60+1) = 0.0320")

---

## 5. Diversity Reranking (MMR)

### 5.1 The Problem with Redundancy

Top results often say the same thing differently:
```
1. Neural networks learn from data.
2. Neural nets are trained on data.
3. Networks learn patterns from datasets.
← All similar!
```

### 5.2 Maximal Marginal Relevance (MMR)

MMR balances relevance with diversity:

$$MMR = \arg\max_{d \in R} \left[ \lambda \cdot Sim(d, q) - (1-\lambda) \cdot \max_{d_s \in S} Sim(d, d_s) \right]$$

Where:
- $d$ = candidate document
- $q$ = query
- $S$ = already selected documents
- $\lambda$ = relevance/diversity trade-off (0.5-0.8 typical)

### 5.3 λ Parameter Effect

| λ Value | Behavior |
|---------|----------|
| 1.0 | Pure relevance (no diversity) |
| 0.5 | Balanced |
| 0.0 | Maximum diversity |

In [ ]:
# Diversity Reranking Example
diversity_reranker = DiversityReranker(lambda_param=0.7)

# Documents with some redundancy
redundant_docs = [
    Document(id="1", content="Neural networks learn patterns from data.", score=0.9),
    Document(id="2", content="Neural nets are trained on datasets to learn.", score=0.88),
    Document(id="3", content="Deep learning uses layers of neurons.", score=0.85),
    Document(id="4", content="Networks learn from examples in data.", score=0.82),
    Document(id="5", content="Computer vision recognizes images.", score=0.75),
]

query = "How do neural networks learn?"

# Apply MMR
diverse_results = diversity_reranker.rerank(query, redundant_docs, top_k=3)

print("Diversity Reranking (MMR) Results")
print("=" * 60)
print(f"λ = 0.7 (70% relevance, 30% diversity)\n")

print("Original Top 3:")
for i, d in enumerate(redundant_docs[:3], 1):
    print(f"  {i}. {d.content}")

print("\nDiversified Top 3:")
for r in diverse_results:
    print(f"  {r.final_rank}. {r.document.content}")
    print(f"     MMR: {r.rerank_score:.3f}")

---

## 6. Building a Reranking Pipeline

### 6.1 Multi-Stage Pipeline

Combine multiple rerankers for best results:

```
Query → BM25 (1000) → Dense (100) → Cross-Encoder (20) → MMR (10)
```

In [ ]:
from src.reranking.reranking import RerankingPipeline

# Build pipeline
pipeline = RerankingPipeline()
pipeline.add_stage(cross_encoder)  # First: accuracy
pipeline.add_stage(diversity_reranker)  # Then: diversity

print("Reranking Pipeline Architecture")
print("=" * 60)
print("""
┌──────────────────┐
│  Initial Results │ (from retrieval)
└────────┬─────────┘
         ▼
┌──────────────────┐
│  Cross-Encoder   │ Stage 1: Improve relevance
│     Reranker     │
└────────┬─────────┘
         ▼
┌──────────────────┐
│    Diversity     │ Stage 2: Add variety
│   Reranker (MMR) │
└────────┬─────────┘
         ▼
┌──────────────────┐
│   Final Results  │ High quality + diverse
└──────────────────┘
""")

### 6.2 Best Practices

| Aspect | Recommendation |
|--------|----------------|
| **Initial retrieval** | Use fast methods (BM25, bi-encoder) |
| **Top-K for reranking** | 50-200 documents |
| **Cross-encoder model** | ms-marco-MiniLM-L-6-v2 (fast), ms-marco-electra-base (accurate) |
| **MMR λ** | Start with 0.7, tune based on use case |
| **Latency budget** | Allocate 60% to retrieval, 40% to reranking |

### 6.3 Common Pitfalls

❌ **Don't:**
- Rerank too many documents (expensive)
- Ignore diversity for QA systems
- Use cross-encoder for initial retrieval

✅ **Do:**
- Monitor reranking latency
- Evaluate with nDCG, MRR
- A/B test reranking strategies

---

## 📝 Summary

### Key Takeaways

1. **Cross-Encoder**: Most accurate, processes query+doc together
2. **LLM Reranking**: Zero-shot, semantic understanding
3. **RRF**: Simple, effective fusion without score normalization
4. **MMR**: Balances relevance with diversity

### When to Use What

| Scenario | Best Method |
|----------|-------------|
| High accuracy needed | Cross-Encoder |
| Multiple retrieval systems | RRF |
| Avoiding redundancy | MMR |
| Limited training data | LLM Reranking |

### Next Steps

1. Try the [Evaluation Module](../week10-evaluation/) to measure improvements
2. Build a complete [RAG Pipeline](../week9-orchestration/) with reranking
3. Explore production deployment in [Week 13](../week13-deployment/)